# unzip the given zip (data) file
# Loading and extracting the data.csv and inspecting the first 5 rows using head method
Load the extracted CSV into a pandas DataFrame and inspect the first few rows

In [6]:
import zipfile
import os

# Ensure the target directory exists or create it if needed
os.makedirs('./data/', exist_ok=True)

# Unzip the dataset into the specified directory
with zipfile.ZipFile('hw0_dataset_1M.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('./data/')

import pandas as pd

# Read the dataset from the CSV file, ensuring bad lines are skipped and optimized for large files
file_path = "./data/hw0_dataset_1M.csv"
data = pd.read_csv(file_path, header=None, names=[f"Column_{i}" for i in range(30)], on_bad_lines='skip', dtype=str, low_memory=True)

# Display the first few rows to preview the structure of the dataset
print(data.head())
print("Dataset has been successfully imported into the DataFrame.")

# Data cleaning: ensure the second column is treated as a string for further processing

# Convert the second column to a string type for string-based operations
data[1] = data[1].astype(str)

# Locate rows where the second column contains the substring 'key', ignoring case sensitivity
key_rows = data[data[1].str.contains('key', case=False, na=False)]

# Show the first few rows that contain the word 'key'
print(key_rows.head())


  Column_0     Column_1         Column_2                 Column_3    Column_4  \
0      NaN          key  doi-asserted-by                      DOI  first-page   
1        0  e_1_2_7_2_1        publisher         10.1038/35041545         NaN   
2        1  e_1_2_7_3_1        publisher  10.1126/science.1155121         NaN   
3        2  e_1_2_7_4_1        publisher    10.1007/s003820050007         NaN   
4        3  e_1_2_7_5_1        publisher     10.1029/2000GL012471         NaN   

       Column_5 Column_6 Column_7 Column_8       Column_9  ... Column_20  \
0  volume-title   author     year   volume  journal-title  ...       NaN   
1           NaN      NaN      NaN      NaN            NaN  ...       NaN   
2           NaN      NaN      NaN      NaN            NaN  ...       NaN   
3           NaN      NaN      NaN      NaN            NaN  ...       NaN   
4           NaN      NaN      NaN      NaN            NaN  ...       NaN   

  Column_21 Column_22 Column_23 Column_24 Column_25 Colu

KeyError: 1

# 1.Converting the second column to string type and search for rows which contains the 'key'
# 2.Saving the cleaned data to cleaned_data.csv

In [ ]:
# Ensure the second column is of type string for uniform processing
df[1] = df[1].astype(str)

# Identify rows in column 1 that contain the word 'key', case insensitive
key_matching_rows = df[df[1].str.contains('key', case=False, na=False)]

# Identify the row with the most non-NaN values, which will be used as the new header
max_non_null_row = key_matching_rows.loc[key_matching_rows.notna().sum(axis=1).idxmax()]

# Set this row as the new header for the DataFrame
df.columns = max_non_null_row

# Remove the rows used for finding the header, as they are no longer needed
df = df.drop(key_matching_rows.index)

# Remove columns with NaN values in the header and drop rows that consist entirely of NaNs
cleaned_df = df.loc[:, df.columns.notna()]
cleaned_df = cleaned_df.dropna(how='all')

# Display the first few rows of the cleaned data
print(cleaned_df.head())

# Save the cleaned DataFrame to a CSV file and print confirmation
df_cleaned.to_csv('data/cleaned_data.csv', index=False)
print("The cleaned data has been saved.")

# Save the selected columns to another CSV file with a new name
output_csv_path = 'data/cleaned_dataset_selected_columns.csv'
cleaned_df.to_csv(output_csv_path, index=False)


# 1.Extract,transform and export JSON data
# 2.Restrict the data to the required 3 columns: 'DOI', 'journal-title', and 'doi-asserted-by'
# 3. Normalizing 
# 4. Saving to data_cleaned_normalized csv
# 5.Saving to data_cleaned_normalized json

In [ ]:
import pandas as pd
import re

df_transformed = df_cleaned[['DOI', 'journal-title', 'doi-asserted-by']].copy()


df_transformed['DOI'] = df_transformed['DOI'].str.lower()
df_transformed['journal-title'] = df_transformed['journal-title'].apply(lambda x: re.sub(r'\s+', ' ', x.strip()) if pd.notnull(x) else x)

df_transformed['journal-title'] = df_transformed['journal-title'].apply(lambda x: re.sub(r'[^\w\s]', '', x) if pd.notnull(x) else x)

import os
os.makedirs('data', exist_ok=True)

#saving to csv
csv_filename = 'data/data_cleaned_normalized.csv'
df_transformed.to_csv(csv_filename, index=False)
#saving to json
json_filename = 'data/data_cleaned_normalized.json'
df_transformed.to_json(json_filename, orient='records', lines=True)
print("Succesful")

# 1.Filter, transform and export CSV data.
# 2. getting journal information

In [ ]:
import pandas as pd
import os

def filter_and_export_data(df, csv_output_path, json_output_path):
    df_filtered = df.dropna(subset=['DOI', 'journal-title'])
    df_filtered_unique = df_filtered.drop_duplicates(subset='DOI')
    print("Columns before selection:", df_filtered_unique.columns)
    df_filtered_final = df_filtered_unique[['DOI', 'journal-title']].copy()  
    print("Columns after selection:", df_filtered_final.columns)
    os.makedirs(os.path.dirname(csv_output_path), exist_ok=True)
    df_filtered_final.to_csv(csv_output_path, index=False)
    df_filtered_final.to_json(json_output_path, orient='records', lines=True)
    print(f"Filtered data saved to '{csv_output_path}' and '{json_output_path}'.")

csv_output_path = 'data/data_filtered.csv'
json_output_path = 'data/data_filtered.json'

filter_and_export_data(df_cleaned, csv_output_path, json_output_path)

def n_most_frequent_doi(df, n):
    doi_counts = df['DOI'].value_counts().head(n)
    return doi_counts.reset_index().rename(columns={'index': 'DOI', 'DOI': 'count'})

def journal_lookup(df, s):
    filtered_df = df[df['journal-title'].str.contains(s, case=False, na=False)]
    return filtered_df


# Get the 5 most frequent DOIs
most_frequent_dois = n_most_frequent_doi(df_cleaned, 5)
print(most_frequent_dois)

# Lookup journals with 'Science' in the title
journal_results = journal_lookup(df_cleaned, 'Science')
print(journal_results)

